In [1]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [5]:
df = pd.read_excel('loan-data.xlsx')

# New Section

In [6]:
df.describe

<bound method NDFrame.describe of        Unnamed: 0       id  member_id  loan_amnt  funded_amnt  \
0               2        2          2        0.0          0.0   
1               3   545583     703644     2500.0       2500.0   
2               4   532101     687836     5000.0       5000.0   
3               5   877788    1092507     7000.0       7000.0   
4               6   875406    1089981     2000.0       2000.0   
...           ...      ...        ...        ...          ...   
38475       38476   849205    1060907     3000.0       3000.0   
38476       38477   852914    1065048    10400.0      10400.0   
38477       38478   519553     671637    16000.0      10550.0   
38478       38479   825638    1034448    10000.0      10000.0   
38479       38480  1029847    1249126     3200.0       3200.0   

       funded_amnt_inv       term  int_rate  installment emp_length  ...  \
0              0.00000  36 months      0.00         0.00   < 1 year  ...   
1           2500.00000  36 months

In [7]:
df.isnull().sum()

Unnamed: 0                    0
id                            0
member_id                     0
loan_amnt                     1
funded_amnt                   1
funded_amnt_inv               1
term                          0
int_rate                      0
installment                   1
emp_length                  993
home_ownership                0
annual_inc                    2
verification_status           0
issue_d                       0
loan_status                   0
purpose                       0
zip_code                      0
addr_state                    0
dti                           0
delinq_2yrs                   1
earliest_cr_line              0
inq_last_6mths                1
mths_since_last_delinq    24363
open_acc                      1
pub_rec                       1
revol_bal                     4
revol_util                   59
total_acc                     1
total_pymnt                   1
total_pymnt_inv               1
total_rec_prncp               1
total_re

In [8]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
38475    False
38476    False
38477    False
38478    False
38479    False
Length: 38480, dtype: bool

In [9]:
df.columns

Index(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'purpose', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'repay_fail'],
      dtype='object')

In [10]:
df.drop(columns=["Unnamed: 0","id","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line",
                 "inq_last_6mths","mths_since_last_delinq","open_acc","pub_rec","revol_bal","revol_util","total_acc",
                 "total_pymnt_inv",],inplace=True)

In [11]:
df.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,...,loan_status,purpose,total_pymnt,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,2,0.0,0.0,0.0,36 months,0.00,0.00,< 1 year,RENT,0.0,...,Charged Off,major_purchase,0.000000,0.00,0.00,2023-01-07,0.00,2023-01-07,2023-01-07,1
1,703644,2500.0,2500.0,2500.0,36 months,13.98,85.42,4 years,RENT,20004.0,...,Does not meet the credit policy. Status:Fully ...,other,3075.291779,2500.00,575.29,2023-07-13,90.85,2023-08-13,2023-06-16,0
2,687836,5000.0,5000.0,5000.0,36 months,15.95,175.67,4 years,RENT,59000.0,...,Charged Off,debt_consolidation,2948.760000,1909.02,873.81,2023-11-11,175.67,NaT,2023-03-12,1
3,1092507,7000.0,7000.0,7000.0,36 months,9.91,225.58,10+ years,MORTGAGE,53796.0,...,Fully Paid,other,8082.391880,7000.00,1082.39,2023-03-14,1550.27,NaT,2023-03-14,0
4,1089981,2000.0,2000.0,2000.0,36 months,5.42,60.32,10+ years,RENT,30000.0,...,Fully Paid,debt_consolidation,2161.663244,2000.00,161.66,2023-02-14,53.12,NaT,2023-06-16,0


In [12]:
df.describe

<bound method NDFrame.describe of        member_id  loan_amnt  funded_amnt  funded_amnt_inv       term  \
0              2        0.0          0.0          0.00000  36 months   
1         703644     2500.0       2500.0       2500.00000  36 months   
2         687836     5000.0       5000.0       5000.00000  36 months   
3        1092507     7000.0       7000.0       7000.00000  36 months   
4        1089981     2000.0       2000.0       2000.00000  36 months   
...          ...        ...          ...              ...        ...   
38475    1060907     3000.0       3000.0       3000.00000  36 months   
38476    1065048    10400.0      10400.0      10400.00000  36 months   
38477     671637    16000.0      10550.0      10531.35818  60 months   
38478    1034448    10000.0      10000.0      10000.00000  36 months   
38479    1249126     3200.0       3200.0       3200.00000  36 months   

       int_rate  installment emp_length home_ownership  annual_inc  ...  \
0          0.00         0.

In [13]:
df.drop(columns=["member_id","loan_amnt","funded_amnt_inv","issue_d"],inplace=True)

In [14]:
df.head()

,funded_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,total_pymnt,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,0.0,36 months,0.00,0.00,< 1 year,RENT,0.0,Not Verified,Charged Off,major_purchase,0.000000,0.00,0.00,2023-01-07,0.00,2023-01-07,2023-01-07,1
1,2500.0,36 months,13.98,85.42,4 years,RENT,20004.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,3075.291779,2500.00,575.29,2023-07-13,90.85,2023-08-13,2023-06-16,0
2,5000.0,36 months,15.95,175.67,4 years,RENT,59000.0,Not Verified,Charged Off,debt_consolidation,2948.760000,1909.02,873.81,2023-11-11,175.67,NaT,2023-03-12,1
3,7000.0,36 months,9.91,225.58,10+ years,MORTGAGE,53796.0,Not Verified,Fully Paid,other,8082.391880,7000.00,1082.39,2023-03-14,1550.27,NaT,2023-03-14,0
4,2000.0,36 months,5.42,60.32,10+ years,RENT,30000.0,Not Verified,Fully Paid,debt_consolidation,2161.663244,2000.00,161.66,2023-02-14,53.12,NaT,2023-06-16,0


In [15]:
df.drop(columns=["last_pymnt_d","last_credit_pull_d","next_pymnt_d"],inplace=True)

In [16]:
df

,funded_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,total_pymnt,total_rec_prncp,total_rec_int,last_pymnt_amnt,repay_fail
0,0.0,36 months,0.00,0.00,< 1 year,RENT,0.0,Not Verified,Charged Off,major_purchase,0.000000,0.00,0.00,0.00,1
1,2500.0,36 months,13.98,85.42,4 years,RENT,20004.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,3075.291779,2500.00,575.29,90.85,0
2,5000.0,36 months,15.95,175.67,4 years,RENT,59000.0,Not Verified,Charged Off,debt_consolidation,2948.760000,1909.02,873.81,175.67,1
3,7000.0,36 months,9.91,225.58,10+ years,MORTGAGE,53796.0,Not Verified,Fully Paid,other,8082.391880,7000.00,1082.39,1550.27,0
4,2000.0,36 months,5.42,60.32,10+ years,RENT,30000.0,Not Verified,Fully Paid,debt_consolidation,2161.663244,2000.00,161.66,53.12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38475,3000.0,36 months,11.99,99.63,3 years,RENT,74250.0,Verified,Fully Paid,credit_card,3586.619764,3000.00,586.62,100.30,0
38476,10400.0,36 months,13.49,352.88,9 years,RENT,62000.0,Source Verified,Fully Paid,credit_card,12703.534030,10400.00,2303.53,393.08,0
38477,10550.0,60 months,14.96,250.77,10+ years,MORTGAGE,95088.0,Source Verified,Fully Paid,debt_consolidation,14202.267530,10550.00,3652.27,5439.96,0
38478,10000.0,36 months,16.89,355.99,1 year,RENT,48720.0,Not Verified,Fully Paid,debt_consolidation,12815.178320,10000.00,2815.18,380.63,0


In [17]:
df.repay_fail.value_counts()

0    32651
1     5829
Name: repay_fail, dtype: int64

In [18]:
df.isnull().sum()

funded_amnt              1
term                     0
int_rate                 0
installment              1
emp_length             993
home_ownership           0
annual_inc               2
verification_status      0
loan_status              0
purpose                  0
total_pymnt              1
total_rec_prncp          1
total_rec_int            1
last_pymnt_amnt          1
repay_fail               0
dtype: int64

In [19]:
df.home_ownership.unique()

array(['RENT', 'MORTGAGE', 'OWN', 'OTHER', 'NONE'], dtype=object)

In [20]:
home_ownership_mapping = {
    'MORTGAGE': 0,
    'RENT': 1,
    'OWN': 2,
    'OTHER': 3,
    'NONE': 4,
    }

# Use the mapping to replace the strings with integers
df['home_ownership'] = df['home_ownership'].replace(home_ownership_mapping)

In [21]:
df.home_ownership.unique()

array([1, 0, 2, 3, 4], dtype=int64)

In [22]:
df.home_ownership

0        1
1        1
2        1
3        0
4        1
        ..
38475    1
38476    1
38477    0
38478    1
38479    1
Name: home_ownership, Length: 38480, dtype: int64

In [23]:
df.term.unique()

array(['36 months', '60 months'], dtype=object)

In [24]:
df.term=df['term'].str.replace("months","").astype("int64")

In [25]:
from pandas.io.formats.format import DataFrameRenderer
DataFrameRenderer

pandas.io.formats.format.DataFrameRenderer

In [26]:
df.term

0        36
1        36
2        36
3        36
4        36
         ..
38475    36
38476    36
38477    60
38478    36
38479    36
Name: term, Length: 38480, dtype: int64

In [27]:
df["verification_status"].unique

<bound method Series.unique of 0           Not Verified
1           Not Verified
2           Not Verified
3           Not Verified
4           Not Verified
              ...       
38475           Verified
38476    Source Verified
38477    Source Verified
38478       Not Verified
38479    Source Verified
Name: verification_status, Length: 38480, dtype: object>

In [28]:
df["verification_status"]=df["verification_status"].replace('Source Verified','Verified')

In [29]:
df.verification_status.unique()

array(['Not Verified', 'Verified'], dtype=object)

In [30]:
df.loan_status.unique()

array(['Charged Off',
       'Does not meet the credit policy. Status:Fully Paid', 'Fully Paid',
       'Current', 'Does not meet the credit policy. Status:Charged Off',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Default'], dtype=object)

In [31]:
df.term

0        36
1        36
2        36
3        36
4        36
         ..
38475    36
38476    36
38477    60
38478    36
38479    36
Name: term, Length: 38480, dtype: int64

In [32]:
df.shape

(38480, 15)

In [33]:
df.loan_status=df.loan_status.str.replace("Does not meet the credit policy. Status:","")

C:\Users\abc\AppData\Local\Temp\ipykernel_7308\1751626574.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loan_status=df.loan_status.str.replace("Does not meet the credit policy. Status:","")


In [34]:
df.loan_status.unique()

array(['Charged Off', 'Fully Paid', 'Current', 'Late (31-120 days)',
       'In Grace Period', 'Late (16-30 days)', 'Default'], dtype=object)

In [35]:
df['loan_status'] = df['loan_status'].replace(['Late (31-120 days)','Late (16-30 days)'], 'Late ')

In [36]:
df.loan_status.unique()

array(['Charged Off', 'Fully Paid', 'Current', 'Late ', 'In Grace Period',
       'Default'], dtype=object)

In [37]:
emp_length_mapping = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
}

# Use the mapping to replace the strings with integers
df['emp_length'] = df['emp_length'].replace(emp_length_mapping)

In [38]:
df['emp_length']

0         0.0
1         4.0
2         4.0
3        10.0
4        10.0
         ... 
38475     3.0
38476     9.0
38477    10.0
38478     1.0
38479     7.0
Name: emp_length, Length: 38480, dtype: float64

In [39]:
df=df.dropna()

In [40]:
df.isnull().sum()

funded_amnt            0
term                   0
int_rate               0
installment            0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
purpose                0
total_pymnt            0
total_rec_prncp        0
total_rec_int          0
last_pymnt_amnt        0
repay_fail             0
dtype: int64

In [41]:
df.shape

(37485, 15)

In [42]:
#test train

In [43]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['repay_fail']),
                                                 df['repay_fail'],
                                                 test_size=0.2,
                                                random_state=42)

In [44]:
trf1 = ColumnTransformer([
    ('ohe_home_ownership_verification_status_loan_status_purpose'
     ,OneHotEncoder(sparse=False,handle_unknown='ignore'),[5,7,8,9,])
],remainder='passthrough')

In [45]:
# train the model
trf2 = DecisionTreeClassifier()

In [46]:
pipe = Pipeline([
                ('trf1',trf1), #list name and object
                ('trf2',trf2)])

In [47]:
# train
pipe.fit(X_train,y_train) #data ko pipe me fit krna he

C:\Users\abc\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_home_ownership_verification_status_loan_status_purpose',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [5, 7, 8, 9])])),
                ('trf2', DecisionTreeClassifier())])

In [48]:
# Code here
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_home_ownership_verification_status_loan_status_purpose',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [5, 7, 8, 9])]),
 'trf2': DecisionTreeClassifier()}

In [49]:
# Predictions
y_pred = pipe.predict(X_test)

In [50]:
#checking our model performence
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

In [51]:
df.shape

(37485, 15)

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6361
           1       1.00      1.00      1.00      1136

    accuracy                           1.00      7497
   macro avg       1.00      1.00      1.00      7497
weighted avg       1.00      1.00      1.00      7497



In [53]:
pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})

,y_test,y_pred
35496,1,1
12645,0,0
10055,0,0
6294,0,0
8819,0,0
...,...,...
8806,0,0
9165,0,0
10965,1,1
5684,0,0


In [54]:
class_0 = df[df.repay_fail==0].sample(5609)
class_1 = df[df.repay_fail==1]

balance_data = pd.concat([class_0 , class_1])
balance_data

,funded_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,total_pymnt,total_rec_prncp,total_rec_int,last_pymnt_amnt,repay_fail
5376,25000.0,60,20.25,665.83,7.0,0,81996.0,Verified,Fully Paid,debt_consolidation,38177.800000,25000.00,13177.80,11548.71,0
27364,6000.0,60,13.06,136.71,10.0,1,110000.0,Verified,Fully Paid,debt_consolidation,8055.810001,6000.00,2055.81,1925.25,0
36724,25000.0,36,16.07,879.85,4.0,0,120000.0,Verified,Fully Paid,debt_consolidation,38079.809150,25000.00,13079.81,7912.97,0
4911,4000.0,36,13.99,136.70,2.0,1,45468.0,Not Verified,Fully Paid,credit_card,4396.533195,4000.00,396.53,172.75,0
32881,10000.0,36,13.16,337.71,2.0,1,33500.0,Verified,Fully Paid,small_business,10619.843050,10000.00,619.84,8932.31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38454,15450.0,36,17.86,557.48,4.0,0,140000.0,Not Verified,Charged Off,debt_consolidation,5571.400000,3501.49,2069.91,557.48,1
38456,12000.0,60,10.99,260.85,10.0,0,55000.0,Verified,Charged Off,major_purchase,9006.520000,5371.48,2697.51,260.85,1
38458,4500.0,36,14.30,154.46,4.0,1,32500.0,Verified,Charged Off,debt_consolidation,1388.790000,951.07,437.72,154.46,1
38471,1500.0,36,16.29,52.96,1.0,1,27000.0,Not Verified,Charged Off,wedding,86.050000,0.00,0.00,0.00,1


In [55]:
trf1 = ColumnTransformer([
    ('ohe_home_ownership_verification_status_loan_status_purpose'
     ,OneHotEncoder(sparse=False,handle_unknown='ignore'),[5,7,8,9,])
],remainder='passthrough')

In [56]:
X_train,X_test,y_train,y_test = train_test_split(balance_data.drop(columns=['repay_fail']),
                                                balance_data['repay_fail'],
                                                 test_size=0.2,
                                                random_state=42)

In [57]:
X_train.shape

(8974, 14)

In [58]:
import tensorflow

In [59]:

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras

In [60]:


  model =keras.Sequential()
  model.add(layers.Dense(100, activation="relu",input_shape=(14,))) #(28*28,))) 1 d array
  #relu and foftmax are activation function
  model.add(layers.Dense(2, activation="sigmoid"))#activation function =linearity to non linear
  #2 layers of sequential (added vertically)
  #layer 1 has 512 neurons
  #layer 2 has 10 neurons
  model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])




In [ ]:
history=model.fit(X_train, y_train, epochs=30,batch_size=2,verbose=1)

In [ ]:
pipe = Pipeline([
                ('trf1',trf1), #list name and object

                    ("model",model()) ])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
# Getting predictions for the test set
y_pred = model.predict_classes(X_test)
# Getting accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
!pip install pycaret #ye sb classifiers ko compare kare ga
#for  machine learning


In [ ]:
!pip install autokeras #for deep learning

In [ ]:
from pycaret.classification import compare_models

In [ ]:
from pycaret.classification import setup

In [ ]:
grid=setup(data=df,target=df.columns[-1])
grid

In [ ]:
best=compare_models()

In [ ]:
import autokeras
from autokeras import StructuredDataClassifier

In [ ]:
search=StructuredDataClassifier(max_trials=15)

In [ ]:
search.fit(x=X_train,y=y_train,verbose=1)